In [1]:
import pandas as pd

In [ ]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List
 
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
 
import fire
import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

## Laptop Stats Dataset Preprocessing

In [ ]:
stats_df = pd.read_csv("archive/laptop_data.csv")
stats_df = stats_df[stats_df["Company"] == "Apple"]

def stats_to_string(df):
    return f"""{df["Company"]} {df["TypeName"]} {df["Inches"]}""", f"""Ноутбук {df["Company"]} {df["TypeName"]} обладает экраном {df["ScreenResolution"]}, процессором {df["Cpu"]}. Объём его оперативной памяти {df["Ram"]}, его жесткий диск - {df["Memory"]}. Также у него есть графический процессор {df["Gpu"]}. Он весит {df["Weight"]}.""", df["Price"]
    

In [ ]:
model_list, info_list, price_list = [], [], []
for idx, row in stats_df.iterrows():
    model, info, price = stats_to_string(row)
    model_list.append(model)
    info_list.append(info)
    price_list.append(price)

In [ ]:
stats_df["Model"] = model_list
stats_df["Info"] = info_list
stats_df["Price"] = price_list

In [ ]:
stats_df.to_csv("datasets/stats_df.csv", index=False)

## HuggingFace Dataset creation

In [50]:
INPUT_QUESTION = [
    "Расскажи мне про",
    "Напиши мне характеристики макбука",
    "Какие параметры у ноутбука",
    "Какие преимущества есть у",
    "Чем может похвастаться ноутбук",
    "Чем хорош макбук",
    "Чем от других ноутбуков отличается",
    "Расскажи мне про хороший ноутбук",
    "Какой ноутбук сейчас может быть очень хорошим?",
    "Что умеет",
    "Почему стоит купить",
    "Какой ноутбук мне порекомендуешь?",
    "Посоветуй мне мощный макбук."
]

In [41]:
def generate_prompt(question, data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Answer question of client about advantages and disadvantages of macbook. Tell him about his goods and recommend him to buy it.
### Input:
{question} {data_point["Model"]}
### Response:
{data_point["Info"]}
Он стоит {data_point["Price"]} рублей."""

In [43]:
df1 = pd.read_csv("datasets/stats_df.csv")
df2 = pd.read_csv("datasets/i-lite.csv")
df3 = pd.read_csv("datasets/istudio-shop.csv")

dfs = [df1, df2, df3]

In [51]:
prompts = []
for cur_df in dfs:
    for _, row in cur_df.iterrows():
        for q in INPUT_QUESTION:
            prompts.append(generate_prompt(q, row))

In [53]:
df = {"prompt": prompts}
df = pd.DataFrame(df)182
df.to_csv("prompts.csv")

In [54]:
df

,prompt
0,"Below is an instruction that describes a task,..."
1,"Below is an instruction that describes a task,..."
2,"Below is an instruction that describes a task,..."
3,"Below is an instruction that describes a task,..."
4,"Below is an instruction that describes a task,..."
...,...
2283,"Below is an instruction that describes a task,..."
2284,"Below is an instruction that describes a task,..."
2285,"Below is an instruction that describes a task,..."
2286,"Below is an instruction that describes a task,..."
